[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nakamura196/ddc-tutorial2025/blob/main/notebooks/tutorial_01_rawgraphs/convert_for_rawgraphs.ipynb)

# NDL SRU検索結果をRAWGraphs用に変換

NDL（国立国会図書館）SRU検索結果CSVをRAWGraphsで可視化するためのデータに変換します。

## 入出力

- **入力**: `ndl_sru_grouped_result.csv`
- **出力**: `ndl_sru_for_rawgraphs.csv`
- **NDCラベル**: `ndc9_class91_hierarchical.csv`（`fetch_ndc_labels.ipynb`で取得）

## 出力フィールド

| フィールド | 説明 |
|-----------|------|
| year | 出版年（1800-1867） |
| title | 作品名 |
| ndc_code | NDCコード（番号のみ） |
| genre | NDCコード:ジャンル名（階層ラベル） |
| genre_major | 大分類のみ（詩歌、小説等） |
| location | 出版地（正規化済み） |
| creator_normalized | 上位著者（その他/不明） |

## 設定

In [1]:
# 年の範囲
MIN_YEAR = 1800
MAX_YEAR = 1867

# 上位著者の数（これ以外は「その他」に集約）
TOP_N_CREATORS = 15

## ライブラリのインポート

In [2]:
import csv
from collections import Counter
from io import StringIO

import pandas as pd  # Google Colabにはプリインストール済み

## データファイルの準備

Google Colabの場合、GitHubからデータをダウンロードします。
ローカル環境の場合は、カレントディレクトリのファイルを使用します。

In [ ]:
# データファイルの準備
import urllib.request

# Google Colabかどうかを判定
try:
    import google.colab
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

if IS_COLAB:
    # Google Colab: GitHubからデータをダウンロード
    BASE_URL = "https://raw.githubusercontent.com/nakamura196/ddc-tutorial2025/refs/heads/main"
    
    # NDCラベルファイルをダウンロード
    ndc_url = f"{BASE_URL}/notebooks/tutorial_01_rawgraphs/ndc9_class91_hierarchical.csv"
    urllib.request.urlretrieve(ndc_url, "ndc9_class91_hierarchical.csv")
    print(f"ダウンロード完了: ndc9_class91_hierarchical.csv")
    
    # 入力データファイルをダウンロード
    data_url = f"{BASE_URL}/data/ndl_sru_grouped_result.csv"
    urllib.request.urlretrieve(data_url, "ndl_sru_grouped_result.csv")
    print(f"ダウンロード完了: ndl_sru_grouped_result.csv")
    
    INPUT_FILE = "ndl_sru_grouped_result.csv"
else:
    # ローカル環境: 相対パスを使用
    print("ローカル環境で実行中。相対パスのファイルを使用します。")
    INPUT_FILE = "../../data/ndl_sru_grouped_result.csv"

## ヘルパー関数の定義

In [4]:
def load_ndc_labels(csv_path: str) -> dict:
    """
    NDCラベルCSVを読み込んで辞書を返す

    Args:
        csv_path: ndc9_class91_hierarchical.csv のパス

    Returns:
        {NDCコード: 階層ラベル} の辞書
    """
    labels = {}
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                code = row['code']
                labels[code] = row['label_hierarchical']
    except FileNotFoundError:
        print(f"警告: {csv_path} が見つかりません。fetch_ndc_labels.ipynb を実行してください。")

    return labels

In [5]:
def ndc_to_genre(ndc: str, ndc_labels: dict) -> tuple:
    """
    単一のNDCコードからジャンル情報を返す

    Returns:
        (ndc_code, genre, genre_major) のタプル
        - ndc_code: NDCコード（番号のみ）
        - genre: NDCコード:階層ラベル
        - genre_major: 大分類のみ
    """
    if not ndc:
        return ("", "不明", "不明")

    ndc = ndc.strip()
    matched_code = ""
    matched_label = ""

    # 完全一致を優先して検索
    if ndc in ndc_labels:
        matched_code = ndc
        matched_label = ndc_labels[ndc]
    else:
        # 前方一致で検索（より具体的なコードを先にチェック）
        for code in sorted(ndc_labels.keys(), key=len, reverse=True):
            if ndc.startswith(code):
                matched_code = code
                matched_label = ndc_labels[code]
                break

    if not matched_code:
        return (ndc, "その他", "その他")

    # 大分類を抽出（階層ラベルの最初の部分）
    genre_major = matched_label.split('/')[0] if matched_label else "その他"

    return (matched_code, f"{matched_code}:{matched_label}", genre_major)

In [6]:
def normalize_location(loc: str) -> str:
    """単一の出版地を正規化する"""
    if not loc:
        return "不明"

    loc = loc.strip()

    # 江戸系
    if any(x in loc for x in ['江戸', '東都', '東京']):
        return "江戸"
    # 京都系
    elif any(x in loc for x in ['京都', '皇都', '平安', '京']):
        return "京都"
    # 大阪系
    elif any(x in loc for x in ['大阪', '大坂', '浪華', '浪花', '坂陽']):
        return "大阪"
    elif '名古屋' in loc:
        return "名古屋"
    elif '長崎' in loc:
        return "長崎"
    elif '金沢' in loc:
        return "金沢"
    elif '出版地不明' in loc:
        return "不明"
    elif loc:
        return "その他"
    else:
        return "不明"

In [7]:
def split_multi_value(value: str, separator: str = ';') -> list:
    """セミコロン区切りの複数値を分割してリストで返す"""
    if not value:
        return ['']
    return [v.strip() for v in value.split(separator)]


def get_first_value(value: str, separator: str = ';') -> str:
    """セミコロン区切りの複数値から最初の値のみを取得"""
    if not value:
        return ''
    return value.split(separator)[0].strip()

In [8]:
def extract_year(issued: str, min_year: int = MIN_YEAR, max_year: int = MAX_YEAR) -> int:
    """issued文字列から年を抽出する（範囲外はNone）"""
    if not issued:
        return None

    year_str = issued[:4]
    try:
        year = int(year_str)
        if min_year <= year <= max_year:
            return year
    except ValueError:
        pass

    return None

In [9]:
def parse_creator_name(creator: str) -> str:
    """単一の著者名から名前部分のみを抽出（生没年を除去）"""
    if not creator:
        return ""
    creator = creator.strip()
    parts = creator.split(',')
    if len(parts) >= 2:
        return f"{parts[0].strip()} {parts[1].strip()}"
    else:
        return creator.split('∥')[0].strip()


def get_top_creators(rows: list, top_n: int = TOP_N_CREATORS) -> set:
    """上位N名の著者名セットを返す（複数著者も全てカウント）"""
    creator_count = Counter()

    for row in rows:
        creators_raw = row.get('creators_name', '')
        if creators_raw:
            for creator in split_multi_value(creators_raw):
                name = parse_creator_name(creator)
                if name:
                    creator_count[name] += 1

    return set(name for name, _ in creator_count.most_common(top_n))


def normalize_creator(creator: str, top_creators: set) -> str:
    """単一の著者名を正規化（上位著者以外は「その他」）"""
    if not creator:
        return "不明"

    name = parse_creator_name(creator)
    if not name:
        return "不明"

    if name in top_creators:
        return name
    else:
        return "その他"

## データの読み込み

In [10]:
# NDCラベルを読み込み
ndc_labels = load_ndc_labels('ndc9_class91_hierarchical.csv')
print(f"NDCラベル: {len(ndc_labels)}件")

NDCラベル: 216件


In [ ]:
# 入力ファイル読み込み
with open(INPUT_FILE, 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

print(f"入力: {len(rows)}件")

In [ ]:
# 上位著者を抽出
top_creators = get_top_creators(rows, TOP_N_CREATORS)
print(f"\n上位{TOP_N_CREATORS}著者:")
for c in sorted(top_creators):
    print(f"  - {c}")

## データ変換

In [ ]:
# 変換処理（複数値は最初の値のみ使用）
output_rows = []
skipped_count = 0

for row in rows:
    # 各フィールドの最初の値を取得
    year = extract_year(get_first_value(row.get('issued', '')))
    if year is None:
        skipped_count += 1
        continue  # 年が取得できない行はスキップ

    # NDC情報を取得（コード、階層ラベル、大分類）
    ndc_code, genre, genre_major = ndc_to_genre(
        get_first_value(row.get('ndc_raw', '')), ndc_labels
    )

    output_rows.append({
        'year': year,
        'title': get_first_value(row.get('title_main', '')),
        'ndc_code': ndc_code,
        'genre': genre,
        'genre_major': genre_major,
        'location': normalize_location(get_first_value(row.get('publisher_location', ''))),
        'creator_normalized': normalize_creator(get_first_value(row.get('creators_name', '')), top_creators)
    })

print(f"変換完了: {len(output_rows)}件（年不明{skipped_count}件除外）")

In [ ]:
# DataFrameに変換
df = pd.DataFrame(output_rows)
df

## 統計情報の表示

In [ ]:
print("=== ジャンル大分類 ===")
genre_major_count = df['genre_major'].value_counts()
for g, c in genre_major_count.items():
    print(f"  {c:4d}: {g}")

In [ ]:
print("=== ジャンル詳細分布 ===")
genre_count = df['genre'].value_counts()
for g, c in genre_count.items():
    print(f"  {c:4d}: {g}")

In [ ]:
print("=== 出版地分布 ===")
loc_count = df['location'].value_counts()
for l, c in loc_count.items():
    print(f"  {c:4d}: {l}")

In [ ]:
print("=== 著者分布 ===")
creator_count = df['creator_normalized'].value_counts()
for cr, c in creator_count.items():
    print(f"  {c:4d}: {cr}")

## CSVファイルとしてダウンロード

In [ ]:
# 全データを保存
output_filename = 'ndl_sru_for_rawgraphs.csv'
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"保存: {output_filename}（{len(df)}件）")

# 1800年除外版を保存
df_filtered = df[df['year'] != 1800]
output_filename_filtered = 'ndl_sru_for_rawgraphs_exclude1800.csv'
df_filtered.to_csv(output_filename_filtered, index=False, encoding='utf-8')
print(f"保存: {output_filename_filtered}（{len(df_filtered)}件、1800年{len(df) - len(df_filtered)}件除外）")

In [ ]:
# Google Colabでファイルをダウンロード
try:
    from google.colab import files
    files.download(output_filename)
    files.download(output_filename_filtered)
    print("ダウンロードを開始しました")
except ImportError:
    print("ローカル環境で実行中。ファイルはカレントディレクトリに保存されています。")